In [1]:
import numpy as np
np.__version__

'1.18.1'

In [2]:
import pandas as pd
pd.__version__

'1.0.1'

In [3]:
import matplotlib
matplotlib.__version__

'3.1.3'

In [4]:
import matplotlib.pyplot as plt

In [5]:
%matplotlib inline

In [6]:
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension
#!jupyter nbextension enable --py gmaps
#!jupyter nbextension enable --py --sys-prefix gmaps
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager
#!jupyter lab build

In [7]:
import gmaps
gmaps.__version__

'0.9.0'

In [8]:
import requests
import os
import pprint as pp
from api_keys import google_api_key

In [51]:
# get the homeless_shelter directory
path = os.path.join(os.getcwd(), 'homeless_shelter')

In [52]:
# walk through the homeless_shelter directory
# to find the file "daily-shelter-occupancy-readme.xlsx"
for dirpath, subdirs, files in os.walk(path):
    subdirs[:] = [d for d in subdirs if d not in '.ipynb_checkpoints']
    for x in files:
        if x.split(".")[1] == "xlsx":
            metadata_path = os.path.join(dirpath, x)

# walk through the homeless_shelter directory
# to find the file paths "daily-shelter-occupancy-20XX-csv.csv"
file_paths = []
for dirpath, subdirs, files in os.walk(path):
    subdirs[:] = [d for d in subdirs if d not in '.ipynb_checkpoints']
    for x in files:
        if x.split(".")[1] == "csv":
            if x != "google_geocode_lats_lngs.csv":
                file_paths.append(os.path.join(dirpath, x))
file_paths.sort()

# metadata_path, a string, holds the pathway to the metadata
# file_paths, a list of strings, holds the pathway to each file

In [53]:
# homeless metadata dataframe
pd.set_option('max_colwidth', 400)
df_metadata = pd.read_excel(metadata_path)
df_metadata

,Field Name,Description,Example
0,Organization_name,Organization Name,The registered name of the non-profit entity responsible \nfor the shelter operation.
1,Shelter_name,Name of the shelter site,"The shelter name generally delineates the subgroup within \nthe organization that is responsible for the shelter. For example the City of Toronto is the organization responsible for the shelters named: Women's Residence, Family Residence, Seaton House etc."
2,Facility_name,Name of the facility,"In some cases shelters share the same name but the actual service is provided in multiple facilities. For example Women's Residence has 3 facilities: the Alexandra Hotel, 63 Bellwoods and 674 Dundas Street West"
3,Program_name,Name of the program within the facility,"Some shelters offer different programs or services within the same \nfacility. For example Seaton House offers emergency shelter on certain floors of 339 George Street but other floors within the same facility are dedicated to longer staying clients. In this example, the program name is the ""Long Term"" program."
4,Occupancy,Number of homeless clients in the program at 4:00 AM,"Occupancy is a point in time number at 4 AM of the next day.\nFor example, the occupancy count of January 1st would be taken on January 2nd at 4 AM."
5,Capacity,Maximum number of spaces available for homeless clients within the program.,"The term ""spaces available"" can mean a bed or a mat/cot. In the \nfamily sector it is possible to exceed available capacity depending on the bed configuration of each room. For example a family of five, who have small children, can elect to be accommodated in a room with four beds. This situation would keep the family united but would exceed the set capacity of the database."
6,Occupancy_date,Date of the data,NaN
7,Shelter_province,Province of the shelter,NaN
8,Sector,Clientele of the shelter,"Sector can be women, men, youth, co-ed, or family."
9,Shelter_city,City of the shelter,NaN


In [12]:
# reset pandas default option
pd.reset_option('max_colwidth')

In [13]:
# read all 3 .csv files with data into a dictionary
df_dict = dict()
for x in file_paths:
    df_dict[x] = pd.read_csv(x, parse_dates=["OCCUPANCY_DATE"])

In [14]:
# get the shape of each dataframe
total_rows = 0
for y in df_dict.values():
    print(y.shape)
    total_rows += y.shape[0]
    
# and the total number of rows
total_rows

(38700, 13)
(37770, 13)
(39446, 13)


115916

In [15]:
# get the columns for each dataframe (3 in total)
for y in df_dict.values():
    print(y.columns)

Index(['_id', 'OCCUPANCY_DATE', 'ORGANIZATION_NAME', 'SHELTER_NAME',
       'SHELTER_ADDRESS', 'SHELTER_CITY', 'SHELTER_PROVINCE',
       'SHELTER_POSTAL_CODE', 'FACILITY_NAME', 'PROGRAM_NAME', 'SECTOR',
       'OCCUPANCY', 'CAPACITY'],
      dtype='object')
Index(['_id', 'OCCUPANCY_DATE', 'ORGANIZATION_NAME', 'SHELTER_NAME',
       'SHELTER_ADDRESS', 'SHELTER_CITY', 'SHELTER_PROVINCE',
       'SHELTER_POSTAL_CODE', 'FACILITY_NAME', 'PROGRAM_NAME', 'SECTOR',
       'OCCUPANCY', 'CAPACITY'],
      dtype='object')
Index(['_id', 'OCCUPANCY_DATE', 'ORGANIZATION_NAME', 'SHELTER_NAME',
       'SHELTER_ADDRESS', 'SHELTER_CITY', 'SHELTER_PROVINCE',
       'SHELTER_POSTAL_CODE', 'FACILITY_NAME', 'PROGRAM_NAME', 'SECTOR',
       'OCCUPANCY', 'CAPACITY'],
      dtype='object')


In [16]:
# prove that all the columns are equivalent
print("2017 and 2018: ",df_dict[file_paths[0]].columns == df_dict[file_paths[1]].columns)
print("2018 and 2019: ", df_dict[file_paths[1]].columns == df_dict[file_paths[2]].columns)
print("2017 and 2019: ", df_dict[file_paths[0]].columns == df_dict[file_paths[2]].columns)

2017 and 2018:  [ True  True  True  True  True  True  True  True  True  True  True  True
  True]
2018 and 2019:  [ True  True  True  True  True  True  True  True  True  True  True  True
  True]
2017 and 2019:  [ True  True  True  True  True  True  True  True  True  True  True  True
  True]


In [17]:
# lets create 1 dataframe from the 3
df = pd.concat([x for x in df_dict.values()])
df.head()

,_id,OCCUPANCY_DATE,ORGANIZATION_NAME,SHELTER_NAME,SHELTER_ADDRESS,SHELTER_CITY,SHELTER_PROVINCE,SHELTER_POSTAL_CODE,FACILITY_NAME,PROGRAM_NAME,SECTOR,OCCUPANCY,CAPACITY
0,1,2017-01-01,COSTI Immigrant Services,COSTI Reception Centre,100 Lippincott Street,Toronto,ON,M5S 2P1,COSTI Reception Centre,COSTI Reception Ctr CITY Program,Co-ed,16,16.0
1,2,2017-01-01,Christie Ossington Neighbourhood Centre,Christie Ossington Men's Hostel,973 Lansdowne Avenue,Toronto,ON,M6H 3Z5,Christie Ossington Men's Hostel,Christie Ossington Extreme Weather Program,Men,13,17.0
2,3,2017-01-01,Christie Ossington Neighbourhood Centre,Christie Ossington Men's Hostel,973 Lansdowne Avenue,Toronto,ON,M6H 3Z5,Christie Ossington Men's Hostel,Christie Ossington Men's Hostel,Men,63,63.0
3,4,2017-01-01,"Christie Refugee Welcome Centre, Inc.",Christie Refugee Welcome Centre,43 Christie Street,Toronto,ON,M6G 3B1,Christie Refugee Welcome Centre,Christie Refugee Welcome Ctr - Settlement and ...,Families,66,70.0
4,5,2017-01-01,City of Toronto,Birchmount Residence,1673 Kingston Road,Toronto,ON,NaN,Birchmount Res 1673 Kingston Rd,Birchmount Residence,Men,58,60.0


In [18]:
# lets get the shape
df.shape

(115916, 13)

In [19]:
# lets get some info about the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115916 entries, 0 to 39445
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   _id                  115916 non-null  int64         
 1   OCCUPANCY_DATE       115916 non-null  datetime64[ns]
 2   ORGANIZATION_NAME    115916 non-null  object        
 3   SHELTER_NAME         115916 non-null  object        
 4   SHELTER_ADDRESS      115916 non-null  object        
 5   SHELTER_CITY         115916 non-null  object        
 6   SHELTER_PROVINCE     115916 non-null  object        
 7   SHELTER_POSTAL_CODE  110422 non-null  object        
 8   FACILITY_NAME        115916 non-null  object        
 9   PROGRAM_NAME         115916 non-null  object        
 10  SECTOR               115916 non-null  object        
 11  OCCUPANCY            115916 non-null  int64         
 12  CAPACITY             115573 non-null  float64       
dtypes: datetime64[n

In [20]:
# some summary statistics?
df.describe()

,_id,OCCUPANCY,CAPACITY
count,115916.000000,115916.000000,115573.000000
mean,19325.915887,56.970211,60.528861
std,11164.582502,83.188821,92.601212
min,1.000000,0.000000,0.000000
25%,9660.000000,13.000000,16.000000
50%,19320.000000,37.000000,40.000000
75%,28979.250000,69.000000,71.000000
max,39446.000000,826.000000,902.000000


In [21]:
# get list of null values
# postal codes are missing
df.isnull().sum()

_id                       0
OCCUPANCY_DATE            0
ORGANIZATION_NAME         0
SHELTER_NAME              0
SHELTER_ADDRESS           0
SHELTER_CITY              0
SHELTER_PROVINCE          0
SHELTER_POSTAL_CODE    5494
FACILITY_NAME             0
PROGRAM_NAME              0
SECTOR                    0
OCCUPANCY                 0
CAPACITY                343
dtype: int64

In [22]:
# the dataset goes by multiple different dates
df["OCCUPANCY_DATE"].sample(10)

26461   2019-09-04
10989   2019-04-13
6254    2018-03-01
28949   2018-10-06
11346   2017-04-18
3421    2018-02-02
21992   2018-07-29
4196    2017-09-02
17385   2018-06-15
36071   2018-12-15
Name: OCCUPANCY_DATE, dtype: datetime64[ns]

In [23]:
# get the minimum date
df["OCCUPANCY_DATE"].min()

Timestamp('2017-01-01 00:00:00')

In [24]:
# get the maximum date
df["OCCUPANCY_DATE"].max()

Timestamp('2019-12-31 00:00:00')

In [25]:
# '_id' column appears to be unique for each row in each dataframe
# but through concatenation we get triplets of each value
df["_id"].unique()

array([    1,     2,     3, ..., 39444, 39445, 39446], dtype=int64)

In [26]:
# lets make a new id and drop _id
# and set id as our index
df["id"] = np.arange(1, len(df)+1)
df = df.drop(columns=["_id"])
df = df.set_index("id")
df.head()

,OCCUPANCY_DATE,ORGANIZATION_NAME,SHELTER_NAME,SHELTER_ADDRESS,SHELTER_CITY,SHELTER_PROVINCE,SHELTER_POSTAL_CODE,FACILITY_NAME,PROGRAM_NAME,SECTOR,OCCUPANCY,CAPACITY
id,,,,,,,,,,,,
1,2017-01-01,COSTI Immigrant Services,COSTI Reception Centre,100 Lippincott Street,Toronto,ON,M5S 2P1,COSTI Reception Centre,COSTI Reception Ctr CITY Program,Co-ed,16,16.0
2,2017-01-01,Christie Ossington Neighbourhood Centre,Christie Ossington Men's Hostel,973 Lansdowne Avenue,Toronto,ON,M6H 3Z5,Christie Ossington Men's Hostel,Christie Ossington Extreme Weather Program,Men,13,17.0
3,2017-01-01,Christie Ossington Neighbourhood Centre,Christie Ossington Men's Hostel,973 Lansdowne Avenue,Toronto,ON,M6H 3Z5,Christie Ossington Men's Hostel,Christie Ossington Men's Hostel,Men,63,63.0
4,2017-01-01,"Christie Refugee Welcome Centre, Inc.",Christie Refugee Welcome Centre,43 Christie Street,Toronto,ON,M6G 3B1,Christie Refugee Welcome Centre,Christie Refugee Welcome Ctr - Settlement and ...,Families,66,70.0
5,2017-01-01,City of Toronto,Birchmount Residence,1673 Kingston Road,Toronto,ON,NaN,Birchmount Res 1673 Kingston Rd,Birchmount Residence,Men,58,60.0


In [27]:
# groupby on "SHELTER_ADDRESS" and "SHELTER_CITY" to get them as a DataFrame
df_locations = df.loc[:, ["SECTOR", "SHELTER_ADDRESS", "SHELTER_CITY"]].groupby(["SHELTER_ADDRESS", "SHELTER_CITY"]).count()
df_locations = df_locations.reset_index().loc[:, ["SHELTER_ADDRESS", "SHELTER_CITY"]]

# this allows us to generate a unique dataframe of addresses and associated cities
df_locations

,SHELTER_ADDRESS,SHELTER_CITY
0,100 Lippincott Street,Toronto
1,101 Ontario Street,Toronto
2,107 Jarvis Street,Toronto
3,1076 Pape Avenue,Toronto
4,1161 Caledonia Road,Toronto
...,...,...
60,87 Pembroke St,Toronto
61,875 Queen St E,Toronto
62,95 Wellesley St. E.,Toronto
63,973 Lansdowne Avenue,Toronto


In [28]:
# create a dictionary that will hold everything
response_dict = {
    "latitudes": [],
    "longitudes": [],
    "formatted_address" :[],
    "postal_code" : [],
    "json_responses" : [], 
    "SHELTER_ADDRESS": df_locations["SHELTER_ADDRESS"].values,
    "SHELTER_CITY": df_locations["SHELTER_CITY"].values
}

# characteristics for google maps geocode api calls
base = "https://maps.googleapis.com/maps/api/geocode/"
outputFormat = "json?"
api_key_phrase = f"&key={google_api_key}"
province = "Ontario"
country = "Canada"

# for loop going over each row of DataFrame
for x in range(len(response_dict["SHELTER_ADDRESS"])):
    # generate the HTML for HTTP API request
    address = response_dict["SHELTER_ADDRESS"][x]
    city = response_dict["SHELTER_CITY"][x]
    HTML_call = (f'{base}{outputFormat}address={address} {city} {province} '+
               f'{country}{api_key_phrase}')
    
    # get the response and turn it from json to a python dictionary
    response = requests.get(HTML_call).json()
    
    # use a try...except statement just in case any of these values cannot be found
    # for a given address
    try:
        response_dict["postal_code"].append(
            # look for types[0] == "postal_code" because some values
            # are missing postal codes
            next(item for item in response["results"][0]["address_components"] 
                if item["types"][0] == "postal_code"
                )["long_name"]
                                         )
        response_dict["latitudes"].append(
            response["results"][0]["geometry"]["location"]["lat"]
                                         )
        response_dict["longitudes"].append(
            response["results"][0]["geometry"]["location"]["lng"]
                                         )
        response_dict["formatted_address"].append(
            response["results"][0]["formatted_address"]
                                         )
        response_dict["json_responses"].append(response)
    except:
        response_dict["latitudes"].append(None)
        response_dict["longitudes"].append(None)
        response_dict["formatted_address"].append(None)
        response_dict["postal_code"].append(None)
        response_dict["json_responses"].append(response)

In [29]:
for index, value in response_dict.items():
    print(index, len(value))

latitudes 65
longitudes 65
formatted_address 65
postal_code 65
json_responses 65
SHELTER_ADDRESS 65
SHELTER_CITY 65


In [30]:
df_lats_longs = pd.DataFrame(response_dict)
df_lats_longs

,latitudes,longitudes,formatted_address,postal_code,json_responses,SHELTER_ADDRESS,SHELTER_CITY
0,43.657677,-79.407047,"100 Lippincott St, Toronto, ON M5S 2P1, Canada",M5S 2P1,{'results': [{'address_components': [{'long_na...,100 Lippincott Street,Toronto
1,43.654558,-79.366575,"101 Ontario St, Toronto, ON M5A 2V2, Canada",M5A 2V2,{'results': [{'address_components': [{'long_na...,101 Ontario Street,Toronto
2,43.652367,-79.372272,"107 Jarvis St, Toronto, ON M5C 2H4, Canada",M5C 2H4,{'results': [{'address_components': [{'long_na...,107 Jarvis Street,Toronto
3,43.690910,-79.349663,"1076 Pape Ave, East York, ON M4K 3W5, Canada",M4K 3W5,{'results': [{'address_components': [{'long_na...,1076 Pape Avenue,Toronto
4,43.715653,-79.466609,"1161 Caledonia Rd, North York, ON M6A 1J8, Canada",M6A 1J8,{'results': [{'address_components': [{'long_na...,1161 Caledonia Road,Toronto
...,...,...,...,...,...,...,...
60,43.659242,-79.372430,"87 Pembroke St, Toronto, ON M5A 2N9, Canada",M5A 2N9,{'results': [{'address_components': [{'long_na...,87 Pembroke St,Toronto
61,43.660007,-79.343222,"875 Queen St E, Toronto, ON M4M 1J2, Canada",M4M 1J2,{'results': [{'address_components': [{'long_na...,875 Queen St E,Toronto
62,43.665864,-79.379215,"95 Wellesley St E, Toronto, ON M4Y 2X9, Canada",M4Y 2X9,{'results': [{'address_components': [{'long_na...,95 Wellesley St. E.,Toronto
63,43.666112,-79.446018,"973 Lansdowne Ave, Toronto, ON M6H 3Z5, Canada",M6H 3Z5,{'results': [{'address_components': [{'long_na...,973 Lansdowne Avenue,Toronto


In [32]:
geocode_df_path = os.path.join(os.path.dirname(file_paths[0]), "google_geocode_lats_lngs.csv")

In [33]:
# save it as a csv
df_lats_longs.to_csv(geocode_df_path, index=False)

In [34]:
df_merge = pd.merge(df, df_lats_longs, on="SHELTER_ADDRESS")
df_merge.head()

,OCCUPANCY_DATE,ORGANIZATION_NAME,SHELTER_NAME,SHELTER_ADDRESS,SHELTER_CITY_x,SHELTER_PROVINCE,SHELTER_POSTAL_CODE,FACILITY_NAME,PROGRAM_NAME,SECTOR,OCCUPANCY,CAPACITY,latitudes,longitudes,formatted_address,postal_code,json_responses,SHELTER_CITY_y
0,2017-01-01,COSTI Immigrant Services,COSTI Reception Centre,100 Lippincott Street,Toronto,ON,M5S 2P1,COSTI Reception Centre,COSTI Reception Ctr CITY Program,Co-ed,16,16.0,43.657677,-79.407047,"100 Lippincott St, Toronto, ON M5S 2P1, Canada",M5S 2P1,{'results': [{'address_components': [{'long_na...,Toronto
1,2017-02-01,COSTI Immigrant Services,COSTI Reception Centre,100 Lippincott Street,Toronto,ON,M5S 2P1,COSTI Reception Centre,COSTI Reception Ctr CITY Program,Co-ed,16,16.0,43.657677,-79.407047,"100 Lippincott St, Toronto, ON M5S 2P1, Canada",M5S 2P1,{'results': [{'address_components': [{'long_na...,Toronto
2,2017-03-01,COSTI Immigrant Services,COSTI Reception Centre,100 Lippincott Street,Toronto,ON,M5S 2P1,COSTI Reception Centre,COSTI Reception Ctr CITY Program,Co-ed,16,16.0,43.657677,-79.407047,"100 Lippincott St, Toronto, ON M5S 2P1, Canada",M5S 2P1,{'results': [{'address_components': [{'long_na...,Toronto
3,2017-04-01,COSTI Immigrant Services,COSTI Reception Centre,100 Lippincott Street,Toronto,ON,M5S 2P1,COSTI Reception Centre,COSTI Reception Ctr CITY Program,Co-ed,16,16.0,43.657677,-79.407047,"100 Lippincott St, Toronto, ON M5S 2P1, Canada",M5S 2P1,{'results': [{'address_components': [{'long_na...,Toronto
4,2017-05-01,COSTI Immigrant Services,COSTI Reception Centre,100 Lippincott Street,Toronto,ON,M5S 2P1,COSTI Reception Centre,COSTI Reception Ctr CITY Program,Co-ed,16,16.0,43.657677,-79.407047,"100 Lippincott St, Toronto, ON M5S 2P1, Canada",M5S 2P1,{'results': [{'address_components': [{'long_na...,Toronto


In [35]:
df_merge.isnull().sum()

OCCUPANCY_DATE            0
ORGANIZATION_NAME         0
SHELTER_NAME              0
SHELTER_ADDRESS           0
SHELTER_CITY_x            0
SHELTER_PROVINCE          0
SHELTER_POSTAL_CODE    5494
FACILITY_NAME             0
PROGRAM_NAME              0
SECTOR                    0
OCCUPANCY                 0
CAPACITY                343
latitudes              4382
longitudes             4382
formatted_address      4382
postal_code            4382
json_responses            0
SHELTER_CITY_y            0
dtype: int64

In [36]:
df_groupby2 = df_merge[["SHELTER_NAME", "FACILITY_NAME", "SHELTER_ADDRESS",
           "SHELTER_CITY_x", "postal_code", "latitudes", "longitudes",
           "OCCUPANCY", "CAPACITY"]]\
    .groupby(["SHELTER_NAME", "FACILITY_NAME", "SHELTER_ADDRESS",
               "SHELTER_CITY_x", "postal_code", "latitudes", "longitudes"])\
    .agg(["mean", "median", "std"]).reset_index()
df_groupby2

SHELTER_NAME                     FACILITY_NAME  \
                                                               
0     Birchmount Residence   Birchmount Res 1673 Kingston Rd   
1       Birkdale Residence                Birkdale Residence   
2       Birkdale Residence    Birkdale Residence Motel (Fam)   
3       Birkdale Residence  Birkdale Residence Motel (Men's)   
4       Birkdale Residence   Birkdale Residence Motel(Co-Ed)   
..                     ...                               ...   
115     YWCA-348 Davenport   YWCA - 348 Davenport Road Women   
116     YWCA-348 Davenport   YWCA - 348 Davenport Road Youth   
117  Youth Without Shelter   Youth w/o Shelter Emerg Shelter   
118  Youth Without Shelter  Youth w/o Shltr Transitional Res   
119      YouthLink Shelter        YouthLink - 747 Warden Ave   

         SHELTER_ADDRESS SHELTER_CITY_x postal_code  latitudes longitudes  \
                                                                            
0     1673 Kingston Road        Toronto     M1N 1S6  43.691569 -79.263923   
1    1229 Ellesmere Road        Toronto     M1P 4V8  43.768282 -79.267447   
2    1229 Ellesmere Road        Toronto     M1P 4V8  43.768282 -79.267447   
3    1229 Ellesmere Road        Toronto     M1P 4V8  43.768282 -79.267447   
4    1229 Ellesmere Road        Toronto     M1P 4V8  43.768282 -79.267447   
..                   ...            ...         ...        ...        ...   
115    348 Davenport Rd.        Toronto     M5R 1K6  43.675403 -79.401348   
116    348 Davenport Rd.        Toronto     M5R 1K6  43.675403 -79.401348   
117   6 Warrendale Court      Etobicoke     M9V 1P9  43.736293 -79.580427   
118   6 Warrendale Court      Etobicoke     M9V 1P9  43.736293 -79.580427   
119       747 Warden Ave        Toronto     M1L 4A8  43.717816 -79.282880   

      OCCUPANCY                       CAPACITY                     
           mean median         std        mean median         std  
0     57.621918   59.0    2.564838   60.000000   60.0    0.000000  
1    173.078539  169.0   12.873355  168.522374  160.0   17.514768  
2    235.522036  172.0  145.775502  289.513346  205.0  174.071090  
3     15.720745   19.0    8.241798   25.821333   36.0   13.605056  
4      1.473469    2.0    0.786874    2.000000    2.0    0.000000  
..          ...    ...         ...         ...    ...         ...  
115   37.446377   39.0    5.413500   38.104348   40.0    5.443217  
116   14.240469   15.0    1.911484   14.835777   15.0    1.235270  
117   32.605479   33.0    0.730360   33.000000   33.0    0.000000  
118   19.550685   20.0    0.723677   19.961644   20.0    0.337196  
119   22.394191   10.0   14.604210   23.595436   10.0   15.145319  

[120 rows x 13 columns]

In [38]:
df_groupby2.columns = ["SHELTER_NAME", "FACILITY_NAME", "SHELTER_ADDRESS",
                            "SHELTER_CITY_x", "postal_code", "latitudes",
                            "longitudes", "OCCUPANCY_mean", "OCCUPANCY_median", "OCCUPANCY_std",
                            "CAPACITY_mean", "CAPACITY_median", "CAPACITY_std"]
df_groupby2

,SHELTER_NAME,FACILITY_NAME,SHELTER_ADDRESS,SHELTER_CITY_x,postal_code,latitudes,longitudes,OCCUPANCY_mean,OCCUPANCY_median,OCCUPANCY_std,CAPACITY_mean,CAPACITY_median,CAPACITY_std
0,Birchmount Residence,Birchmount Res 1673 Kingston Rd,1673 Kingston Road,Toronto,M1N 1S6,43.691569,-79.263923,57.621918,59.0,2.564838,60.000000,60.0,0.000000
1,Birkdale Residence,Birkdale Residence,1229 Ellesmere Road,Toronto,M1P 4V8,43.768282,-79.267447,173.078539,169.0,12.873355,168.522374,160.0,17.514768
2,Birkdale Residence,Birkdale Residence Motel (Fam),1229 Ellesmere Road,Toronto,M1P 4V8,43.768282,-79.267447,235.522036,172.0,145.775502,289.513346,205.0,174.071090
3,Birkdale Residence,Birkdale Residence Motel (Men's),1229 Ellesmere Road,Toronto,M1P 4V8,43.768282,-79.267447,15.720745,19.0,8.241798,25.821333,36.0,13.605056
4,Birkdale Residence,Birkdale Residence Motel(Co-Ed),1229 Ellesmere Road,Toronto,M1P 4V8,43.768282,-79.267447,1.473469,2.0,0.786874,2.000000,2.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,YWCA-348 Davenport,YWCA - 348 Davenport Road Women,348 Davenport Rd.,Toronto,M5R 1K6,43.675403,-79.401348,37.446377,39.0,5.413500,38.104348,40.0,5.443217
116,YWCA-348 Davenport,YWCA - 348 Davenport Road Youth,348 Davenport Rd.,Toronto,M5R 1K6,43.675403,-79.401348,14.240469,15.0,1.911484,14.835777,15.0,1.235270
117,Youth Without Shelter,Youth w/o Shelter Emerg Shelter,6 Warrendale Court,Etobicoke,M9V 1P9,43.736293,-79.580427,32.605479,33.0,0.730360,33.000000,33.0,0.000000
118,Youth Without Shelter,Youth w/o Shltr Transitional Res,6 Warrendale Court,Etobicoke,M9V 1P9,43.736293,-79.580427,19.550685,20.0,0.723677,19.961644,20.0,0.337196


In [39]:
info_box_template = """
<dl>
<dt>Facility Name</dt><dd>{FACILITY_NAME}</dd>
<dt>Shelter Name</dt><dd>{SHELTER_NAME}</dd>
<dt>Address</dt><dd>{SHELTER_ADDRESS}</dd>
<dt>Median Occupancy</dt><dd>{OCCUPANCY_median}</dd>
<dt>Median Capacity</dt><dd>{CAPACITY_median}</dd>
</dl>
"""

homeless_info = [info_box_template.format(**row) for index, row in df_groupby2.iterrows()]
locations = df_groupby2[["latitudes", "longitudes"]]

In [40]:
gmaps.configure(api_key=google_api_key)

In [41]:
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=homeless_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))